In [ ]:
# Leaf Sorting Tree Explorer
import os
import subprocess
import time
from pathlib import Path
from typing import List, Optional, Tuple

# Configuration
VARIANT = 'atomic'
STALEMATE = 'draw'
ENGINE = '/content/stockfish'
THREADS = 4
HASH = 8192
MULTIPV = 10
DEPTH = 13
THRESHOLD = 100

# Colab setup
from google.colab import drive
drive.mount('/content/drive/')

DIR = f'/content/drive/MyDrive/trees/{VARIANT}/'
os.makedirs(DIR, exist_ok=True)

# Check for variants.ini
INI = None
if os.path.exists('/content/drive/MyDrive/trees/variants.ini'):
    os.system('cp /content/drive/MyDrive/trees/variants.ini /content/')
    INI = '/content/variants.ini'

# Check for NNUE
NNUE = None
for file in Path(DIR).glob('*.nnue'):
    os.system(f'cp {file} /content/')
    NNUE = f'/content/{file.name}'
    break

# Data file
DATA = f'{VARIANT}_sorting.txt'
if os.path.exists(f'{DIR}{DATA}'):
    os.system(f'cp {DIR}{DATA} /content/{DATA}')

# Check for engine
if os.path.exists('/content/drive/MyDrive/trees/stockfish'):
    os.system('cp /content/drive/MyDrive/trees/stockfish /content/')
    os.system('chmod +x /content/stockfish')
else:
    os.system('git clone https://github.com/fairy-stockfish/Fairy-Stockfish.git /content/Fairy-Stockfish/')
    os.system('cd /content/Fairy-Stockfish/src/ && make clean && make ARCH=x86-64-modern COMP=gcc all -j2')
    os.system('cp /content/Fairy-Stockfish/src/stockfish /content/')
    os.system('cp /content/stockfish /content/drive/MyDrive/trees/')

In [ ]:
class Engine:
    def __init__(self):
        self.process = subprocess.Popen(
            [ENGINE],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=0
        )
        self.send('uci')
        self.receive('uciok')
        if INI:
            self.send(f'load {INI}')
        self.send(f'setoption name UCI_Variant value {VARIANT}')
        if NNUE:
            self.send(f'setoption name EvalFile value {NNUE}')
            self.send('setoption name Use NNUE value true')
        else:
            self.send('setoption name Use NNUE value false')
        self.send(f'setoption name Threads value {THREADS}')
        self.send(f'setoption name Hash value {HASH}')
        self.send(f'setoption name MultiPV value {MULTIPV}')

    def send(self, command: str):
        if self.process.poll() is None:
            self.process.stdin.write(f'{command}\n')
            self.process.stdin.flush()

    def receive(self, terminator: str) -> List[str]:
        lines = []
        while True:
            line = self.process.stdout.readline().strip()
            if not line and self.process.poll() is not None:
                break
            lines.append(line)
            if terminator in line:
                break
        return lines

    def get_fen(self, command: str = 'position startpos') -> str:
        self.send(command)
        self.send('d')
        lines = self.receive('Sfen:')
        for line in lines:
            if line.startswith('Fen:'):
                return line.split('Fen:', 1)[1].strip()

    def quit(self):
        if self.process.poll() is None:
            self.send('quit')
            self.process.terminate()
            self.process.wait()

class Tree:
    def __init__(self):
        self.engine = Engine()
        self.fen = []
        self.move = []
        self.score = []
        self.best = []
        self.children = []
        self.length = []
        self.fen_to_index = {}
        self.load_data()
        self.last_saved = time.time()
        self.analyzing_white = True
        self.cycle_count = 0

    def load_data(self):
        if not os.path.exists(f'/content/{DATA}'):
            root_fen = self.engine.get_fen()
            self.fen.append(root_fen)
            self.move.append(None)
            self.score.append(0)
            self.best.append(None)
            self.children.append([])
            self.length.append(0)
            self.fen_to_index[f'None|{root_fen}'] = 0
        else:
            with open(f'/content/{DATA}', 'r') as f:
                for line in f:
                    line = line.strip()
                    if not line or line.startswith('%'):
                        continue
                    parts = line.rstrip(';').split(';')
                    if len(parts) < 2:
                        continue
                    header = parts[0].strip().split('|')
                    idx = int(header[0])
                    move = header[1] if header[1] != 'None' else None
                    score_val = int(header[2])
                    best_idx = int(header[3]) if header[3] != 'None' else None
                    length_val = int(header[4])
                    fen_str = parts[1].strip()
                    children_list = []
                    if len(parts) > 2 and parts[2].strip():
                        children_list = [int(x) for x in parts[2].strip().split(',')]
                    self.fen.append(fen_str)
                    self.move.append(move)
                    self.score.append(score_val)
                    self.best.append(best_idx)
                    self.children.append(children_list)
                    self.length.append(length_val)
                    if move:
                        key = f'{move}|{fen_str}'
                    else:
                        key = f'None|{fen_str}'
                    self.fen_to_index[key] = idx
            self.minimax()
        self.root_is_white = 'w' in self.fen[0].split()[1]

    def save_data(self):
        lines = ['% idx|move|score|best|length; fen; children;']
        for idx in range(len(self.fen)):
            move = self.move[idx] if self.move[idx] else 'None'
            best = self.best[idx] if self.best[idx] is not None else 'None'
            header = f'{idx}|{move}|{self.score[idx]}|{best}|{self.length[idx]}'
            children_str = ','.join(map(str, self.children[idx])) if self.children[idx] else ''
            lines.append(f'{header}; {self.fen[idx]}; {children_str};')
        with open(f'/content/{DATA}', 'w') as f:
            f.write('\n'.join(lines) + '\n')
        os.system(f'cp /content/{DATA} {DIR}{DATA}')
        self.last_saved = time.time()
        print('\nSaved',len(self.fen),'positions')

    def minimax(self):
        indices = sorted(range(len(self.fen)),
                        key=lambda i: sum(1 for c in self.children[i]))
        change = True
        while change:
            change = False
            for idx in indices:
                if not self.children[idx]:
                    continue
                old_best = self.best[idx]
                old_score = self.score[idx]
                self.best[idx] = None
                self.score[idx] = -30000
                # Find best child
                for child in self.children[idx]:
                    child_score = -self.score[child]
                    if child_score > self.score[idx]:
                        self.best[idx] = child
                        self.length[idx] = self.length[child] + 1
                        # Tempo degradation
                        if child_score > 0:
                            self.score[idx] = child_score - 1
                        else:
                            self.score[idx] = child_score + 1
                if old_best != self.best[idx] or old_score != self.score[idx]:
                    change = True
            if not change:
                break

    def collect_leaves(self, node: int, analyzing_white: bool,
                      path: List[int] = None,
                      best_score: int = None) -> List[Tuple[int, int, List[int]]]:
        if path is None:
            path = [node]
            best_node = node
            while self.best[best_node] is not None:
                best_node = self.best[best_node]
            best_turn = 'w' in self.fen[best_node].split()[1]
            best_score = self.score[best_node] if best_turn == analyzing_white else -self.score[best_node]
        if not self.children[node]:
            current_turn = 'w' in self.fen[node].split()[1]
            score = self.score[node] if current_turn == analyzing_white else -self.score[node]
            return [(score, node, path)] if best_score - score < THRESHOLD else []
        leaves = []
        current_turn = 'w' in self.fen[node].split()[1]
        if current_turn == analyzing_white:
            for child in self.children[node]:
                leaves.extend(self.collect_leaves(child, analyzing_white, path + [child], best_score))
        elif self.best[node] is not None:
            leaves.extend(self.collect_leaves(self.best[node], analyzing_white, path + [self.best[node]], best_score))
        return leaves

    def multipv(self, position_idx: int):
        self.engine.send(f'position fen {self.fen[position_idx]}')
        self.engine.send(f'go depth {DEPTH}')
        lines = self.engine.receive('bestmove')
        for line in reversed(lines):
            if 'multipv' not in line:
                continue
            parts = line.split()
            alt = int(parts[parts.index('multipv') + 1])
            move = parts[parts.index('pv') + 1]
            score_type = parts[parts.index('score') + 1]
            score = int(parts[parts.index('score') + 2])
            # Convert mate score to centipawns
            if score_type == 'mate':
                if STALEMATE != 'win':
                    if score > 0:
                        score = 30001 - 2 * score
                    else:
                        score = -30000 - 2 * score
                else:
                    if score > 0:
                        score = 30000 - 2 * score
                    elif score < 0:
                        score = -30001 + 2 * score
            child_fen = self.engine.get_fen(f'position fen {self.fen[position_idx]} moves {move}')
            child_key = f'{move}|{child_fen}'
            if child_key in self.fen_to_index:
                child_idx = self.fen_to_index[child_key]
            else:
                child_idx = len(self.fen)
                self.fen_to_index[child_key] = child_idx
                self.fen.append(child_fen)
                self.move.append(move)
                self.score.append(-score)  # Negamax
                self.best.append(None)
                self.children.append([])
                self.length.append(0)
            if child_idx not in self.children[position_idx]:
                self.children[position_idx].append(child_idx)
            if alt == 1:
                break

    def analyze(self):
        self.cycle_count += 1
        side = 'White' if self.analyzing_white else 'Black'
        print('\nCycle:', self.cycle_count, side)

        leaves = self.collect_leaves(0, self.analyzing_white)
        leaves.sort(reverse=True, key=lambda x: x[0])
        print('Found:', len(leaves), 'leaves')
        if leaves:
            print('Score range:', leaves[0][0], 'to', leaves[-1][0])
            for score, leaf_idx, path in leaves:
                self.multipv(leaf_idx)
        self.minimax()
        best_line = []
        node = 0
        while self.best[node] is not None:
            node = self.best[node]
            best_line.append(self.move[node])
        print('Best line:', ' '.join(best_line))
        print('Root eval:', self.score[0])
        self.analyzing_white = not self.analyzing_white
        if time.time() - self.last_saved > 300:
            self.save_data()

    def explore(self):
        try:
            print('Leaf Sorting Tree Explorer')
            print('Variant:', VARIANT)
            print('Depth:', DEPTH)
            print('MultiPV:', MULTIPV)
            print('Threshold:', THRESHOLD)
            print('Loaded', len(self.fen), 'positions')

            while True:
                self.analyze()

        except KeyboardInterrupt:
            print('\nStopped by user')
        except Exception as e:
            print('\nError:', e)
            import traceback
            traceback.print_exc()
        finally:
            self.save_data()
            if hasattr(self, 'engine'):
                self.engine.quit()

In [ ]:
tree = Tree()
tree.explore()